<div style="font-size:12pt;color:#999999">plots for:</div>
<div style="font-size:14pt;color:#000000">Denman DJ, Siegle JS, Koch C, Reid CR, Blanche TJ</div>
<div style="font-size:14pt;color:#000000"><strong>Spatial organization of chromatic pathways in the mouse dorsal lateral geniculate nucleus</strong></div>
<div style="font-size:14pt;color:#000000">Journal of Neuroscience, 2016</div>
<br></br>
<div style="font-size:12pt;color:#000000">all neural data is available in <a href="http://nwb.org/resources/#Format">Neurodata Without Borders format</a> from <a href="http://www.brain-map.org">brain-map.org</a></div>
<div style="font-size:12pt;color:#000000">custom functions available on <a href="https://github.com/">GitHub</a></div>


###imports and magics for plotting in this Jupyter notebook:

In [ ]:
from djd.color_functions import * # functions writted by Daniel J. Denman for analysis and plotting of the data collected for this manuscript
                                  # this import includes associated packages like numpy, csv, h5py, etc... for a full list, 
                                  # see color_functions.py from the above GitHub page
%matplotlib inline

<div style="font-size:14pt;color:#000000">run this cell to hide the inputs and just see the plots. </div>
<div style="font-size:10pt;color:#000000">if you want to see the code, use Cell>All Output>Clear</div>

In [2]:
import hide_code.hide_code as hc
dir = "/Users/administrator/.jupyter/nbconfig"
hc.install(dir)

Starting hide_code.js install...
No config directories contain "custom" folder. Trying site-packages...
Using argument supplied path: /Users/administrator/.jupyter/nbconfig
Copying hide_code.js to /Users/administrator/.jupyter/nbconfig
Attempting to configure custom.js to auto-load hide_code.js...
Custom.js not in custom directory.
Attempting to configure auto-loading for hide_code export handlers.
Jupyter already configured to auto-load export handlers.


<br></br>
<br></br>
###first, load the neural data from NWB files

In [ ]:
path_to_data = '/Users/administrator/Desktop/data/color_data/nwbs/' #<-- Replace this with a local path to the downloaded data. 
                                                 # '/Users/administrator/Desktop/data/color_data/nwbs/' 
                                                 # '/Users/danieljdenman/Desktop/data/color_data/nwbs/' 
        
data={}
for nwb_path in glob.glob(path_to_data+'*.nwb'):#get all NWB files in the folder
    data[os.path.basename(nwb_path).split('_')[0]]=h5py.File(nwb_path)
    
    #do some light conditioning because of mismatches in naming conventions between experiments...
    nwbdata=data[os.path.basename(nwb_path).split('_')[0]]
    if 'flash_green' not in nwbdata['stimulus']['presentation'].keys():
        if 'vis_flash' in nwbdata['stimulus']['presentation'].keys():
            #nwb_file = nwbtools.ExtracellularEphysFile(modify=True)
            nwbdata['stimulus']['presentation']['flash_green']=nwbdata['stimulus']['presentation']['vis_flash']
        if 'flashes_vis' in nwbdata['stimulus']['presentation'].keys():
            nwbdata['stimulus']['presentation']['flash_green']=nwbdata['stimulus']['presentation']['flashes_vis']
    
    if 'flash_uv' not in nwbdata['stimulus']['presentation'].keys():
        if 'uv_flash' in nwbdata['stimulus']['presentation'].keys():
            nwbdata['stimulus']['presentation']['flash_uv']=nwbdata['stimulus']['presentation']['uv_flash']
        if 'flashes_uv' in nwbdata['stimulus']['presentation'].keys():
            nwbdata['stimulus']['presentation']['flash_uv']=nwbdata['stimulus']['presentation']['flashes_uv']
            
    if 'binary_green' not in nwbdata['stimulus']['presentation'].keys():
        if 'green_binary' in nwbdata['stimulus']['presentation'].keys():
            nwbdata['stimulus']['presentation']['binary_green']=nwbdata['stimulus']['presentation']['green_binary']
            nwbdata['stimulus']['presentation']['binary_uv']=nwbdata['stimulus']['presentation']['uv_binary']

<div style="font-size:12pt;color:#000000">create a pandas dataframe, with a row for each cell</div>
<div style="font-size:12pt;color:#000000">this dataframe will be populated at each figure with metrics calculated for each cell.</div>

In [ ]:
#create a pandas dataframe, with a row for each cell
#this dataframe will be populated at each figure with metrics calculated for each cell.
dataframedict_expt = []
dataframedict_cell = []
for expt in data.keys():
    expt_expt=[];expt_cell=[]
    for cell in cell_numbers[expt]['lgn_list_ex']:
        expt_expt.extend([expt])
        expt_cell.extend([cell])
    dataframedict_expt.extend(expt_expt)
    dataframedict_cell.extend(expt_cell)
    
dataframedict={'Experiment':dataframedict_expt,'Cell':dataframedict_cell}
df = pd.DataFrame(dataframedict)

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
   <div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 1</font></strong></div>
   <br></br>
   <div style="font-size:12pt;color:#000000"> load the dome stimulation system measurements</div>

In [ ]:
import djd.jeti as jeti
warp_data = pkl.load(open('/Users/danieljdenman/Academics/allen/BlancheLab/dome/lightMeasure/uniformity/uniformityData.pkl')) #from measurements of projected squares on spherical surface, made manually with a custom protractor.
projector1_data = jeti.jetitodict('/Users/danieljdenman/Desktop/data/color_data/dome/uvdome_withluminanceadjustements.csv') #measured with SpectroCal
projector2_data_g = jeti.jetitodict('/Users/danieljdenman/Desktop/data/color_data/dome/luminance_uvProjector2/green_set2.csv')
projector2_data_u = jeti.jetitodict('/Users/danieljdenman/Desktop/data/color_data/dome/luminance_uvProjector2/uv_set1.csv')

<div style="font-size:12pt;color:#000000"> generate plot of warp accuracy, Figure 1B</div>

In [ ]:
plt.imshow(warp_data['uniformity_post'],clim=(1200-0.2 * 1200,1200+0.2*1200),interpolation='gaussian')

<div style="font-size:12pt;color:#000000"> generate plot of contrast linearity, Figure 1C</div>

In [ ]:
wavelengths = projector2_data_u['wavelengths']
contrasts = np.linspace(10,100,10)
total_irradiance_uv = []
total_irradiance_green = []
total_irradiance = []
for contrast in contrasts:
    both = projector2_data_u['uv_'+str(int(contrast))]+projector2_data_g['green_'+str(int(contrast))]
    total_irradiance.append(np.trapz(both[:300]))
    total_irradiance_uv.append(np.trapz(projector2_data_u['uv_'+str(int(contrast))][:300]))
    total_irradiance_green.append(np.trapz(projector2_data_g['green_'+str(int(contrast))][:300]))
plt.plot(contrasts,total_irradiance,'--o',color='k')
plt.plot(contrasts,total_irradiance_uv,'--o',color='#4B0082')
plt.plot(contrasts,total_irradiance_green,'--o',color='#6B8E23')
plt.xlabel('contrast')
plt.ylabel('radiance ^ 2')

<div style="font-size:12pt;color:#000000"> generate plots of LED irradiance and mouse, Figure 1D</div>

In [ ]:
wv = np.linspace(350,700,350)
mso = np.zeros(350);mso[:249]=jeti.mso
green1_max = np.zeros(350);green1_max=projector1_data['structuredlight_G_100'][:350]
green2_max = np.zeros(350);green2_max[30:]=projector2_data_g['green_100'][:320]
uv1_max = np.zeros(350);uv1_max=projector1_data['structuredlight_UV_100'][:350]
uv2_max = np.zeros(350);uv2_max[30:]=projector2_data_u['uv_100'][:320]
f,ax = plt.subplots(1)
ax.plot(np.linspace(350,700,350),mso/2500,color='r')
axR=ax.twinx()
axR.plot(np.linspace(350,700,350),green1_max,color='#6B8E23',ls='--')
ax.plot(np.linspace(350,700,350),green2_max,color='#6B8E23')
axR.plot(np.linspace(350,700,350),uv1_max,color='#4B0082',ls='--')
ax.plot(np.linspace(350,700,350),uv2_max,color='#4B0082')
plt.xlabel('wavelength (nm)')
plt.ylabel('radiance W/(sr*m2) or normalized sensitivity')
plt.title('LEDs and Mouse Standard Observer')

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 2</font></strong></div>

#### starting from the spike times, generate PSTHs from flashed stimuli and make some measurements on them 

In [ ]:


flash_green_ON_peak=[]; flash_green_ON_area=[];flash_green_ON_latency=[];flash_green_OFF_peak=[];flash_green_OFF_area=[];flash_green_OFF_latency=[]
flash_UV_ON_peak=[];flash_UV_ON_area=[];flash_UV_ON_latency=[];flash_UV_OFF_peak=[];flash_UV_OFF_area=[];flash_UV_OFF_latency=[];flash_luminance_class=[];
flash_bgi_dark=[];flash_bgi_bright=[];flash_bgi_best=[];flash_bgi2_dark=[];flash_bgi2_bright=[];flash_bgi2_best=[]
latency_label = np.ones(np.shape(df)[0])-2
on_latency = [];off_latency = []
flash_green_ON = [];flash_green_OFF = [];flash_UV_ON = [];flash_UV_OFF = [];

pre=0.25
post=1.5
binsize=0.01

for i in range(np.shape(df)[0]):
    try:
        nwb_data = data[df['Experiment'][i]]
        cell = str(str(df['Cell'][i]))
        
        #---------------------------------------------------------------------------------------------------------------------------------------
        #calculate the PSTH for each cell, for each color, for increments and decrements 
        psth = psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                         np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[1::2],
                         binsize=binsize,pre=pre,post=post,timeDomain=True,output='hist')
        flash_green_ON_area.extend([psth_area(psth,pre=pre,binsize=binsize)])
        flash_green_ON_peak.extend([np.max(psth[0])])
        flash_green_ON_latency.extend([psth_latency(psth[0][5:60],psth[1][5:60],smooth=True,offset=8.33)])
        flash_green_ON.extend([psth])

        psth = psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                         np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[2::2],
                         binsize=binsize,pre=pre,post=post,timeDomain=True,output='hist')
        flash_green_OFF_area.extend([psth_area(psth,pre=pre,binsize=binsize)])
        flash_green_OFF_peak.extend([np.max(psth[0])])
        flash_green_OFF_latency.extend([psth_latency(psth[0][5:60],psth[1][5:60],smooth=True,offset=16.66)])
        flash_green_OFF.extend([psth])

        psth = psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                         np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[1::2],
                         binsize=binsize,pre=pre,post=post,timeDomain=True,output='hist')
        flash_UV_ON_area.extend([psth_area(psth,pre=pre,binsize=binsize)])
        flash_UV_ON_peak.extend([np.max(psth[0])])
        flash_UV_ON_latency.extend([psth_latency(psth[0][5:60],psth[1][5:60],smooth=True,offset=8.33)])
        flash_UV_ON.extend([psth])

        psth = psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                         np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[2::2],
                         binsize=binsize,pre=pre,post=post,timeDomain=True,output='hist')
        flash_UV_OFF_area.extend([psth_area(psth,pre=pre,binsize=binsize)])
        flash_UV_OFF_peak.extend([np.max(psth[0])])
        flash_UV_OFF_latency.extend([psth_latency(psth[0][5:60],psth[1][5:60],smooth=True,offset=16.66)])
        flash_UV_OFF.extend([psth])
        #---------------------------------------------------------------------------------------------------------------------------------------
    except: #if there was some problem, populate with np.nan
        print str('FAEL PSTH: ') + df['Experiment'][i] + ' '+str(str(df['Cell'][i]))
        for d in [flash_green_ON_peak,flash_green_ON_area,flash_green_ON_latency,flash_green_OFF_peak,flash_green_OFF_area,flash_green_OFF_latency,flash_UV_ON_peak,
                  flash_UV_ON_area,flash_UV_ON_latency,flash_UV_OFF_peak,flash_UV_OFF_area,flash_UV_OFF_latency,
                 flash_green_ON,flash_green_OFF,flash_UV_ON,flash_UV_OFF]:
            d.extend([np.nan])
df['flash_green_ON_peak']=flash_green_ON_peak
df['flash_green_ON_area']=flash_green_ON_area
df['flash_green_ON_latency']=flash_green_ON_latency
df['flash_green_OFF_peak']=flash_green_OFF_peak
df['flash_green_OFF_area']=flash_green_OFF_area
df['flash_green_OFF_latency']=flash_green_OFF_latency
df['flash_UV_ON_peak']=flash_UV_ON_peak
df['flash_UV_ON_area']=flash_UV_ON_area
df['flash_UV_ON_latency']=flash_UV_ON_latency
df['flash_UV_OFF_peak']=flash_UV_OFF_peak
df['flash_UV_OFF_area']=flash_UV_OFF_area
df['flash_UV_OFF_latency']=flash_UV_OFF_latency
df['flash_green_OFF']=flash_green_OFF
df['flash_green_ON']=flash_green_ON
df['flash_UV_OFF']=flash_UV_OFF
df['flash_UV_ON']=flash_UV_ON

for i in range(np.shape(df)[0]):
    try:
        nwb_data = data[df['Experiment'][i]]
        cell = str(str(df['Cell'][i]))        
        #---------------------------------------------------------------------------------------------------------------------------------------
        #find the best stimulus for this cell 
        maxi = 0
        for stim in ['UV_OFF','UV_ON','green_OFF','green_ON']:
            if df['flash_'+stim+'_peak'][i] > maxi:
                maxi = df['flash_'+stim+'_peak'][i]
                max_type = stim
        
        #using the color of the best stimulus, compare ON and OFF and put this cell in the appropriate class
        #uses a difference of 25msec [half of the duration] in latency to do the classification.
        if  df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i] > 25.:
            latency_label[i]=2
            flash_luminance_class.extend(['ON'])
        else:
            if  df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i] < -25.:
                latency_label[i]=0
                flash_luminance_class.extend(['OFF'])
            else:
                if np.abs( df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i]) < 25.:
                    latency_label[i]=1
                flash_luminance_class.extend(['ON/OFF'])
        
        #using the color of the best stimulus, find the ON and OFF latencies
        on_latency.extend([df['flash_'+max_type.split('_')[0]+'_ON_latency'][i]])
        off_latency.extend([df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i]]) 
        #---------------------------------------------------------------------------------------------------------------------------------------
        
        #---------------------------------------------------------------------------------------------------------------------------------------
        #calculate the UVGi for this cell
        flash_bgi_dark.extend([(df['flash_green_OFF_peak'][i]-df['flash_UV_OFF_peak'][i])/float(df['flash_green_OFF_peak'][i]+df['flash_UV_OFF_peak'][i])])
        flash_bgi_bright.extend([(df['flash_green_ON_peak'][i]-df['flash_UV_ON_peak'][i])/float(df['flash_green_ON_peak'][i]+df['flash_UV_ON_peak'][i])])
        flash_bgi2_dark.extend([(df['flash_green_OFF_area'][i]-df['flash_UV_OFF_area'][i])/float(df['flash_green_OFF_area'][i]+df['flash_UV_OFF_area'][i])])
        flash_bgi2_bright.extend([(df['flash_green_ON_area'][i]-df['flash_UV_ON_area'][i])/float(df['flash_green_ON_area'][i]+df['flash_UV_ON_area'][i])])    
            
        if latency_label[i] == 0:#OFF
            if df['flash_green_OFF_peak'][i] > (np.mean(df['flash_green_OFF'][i][0][0:5])+np.std(df['flash_green_OFF'][i][0][0:5])*3) and df['flash_UV_OFF_peak'][i] > (np.mean(df['flash_UV_OFF'][i][0][0:5])+np.std(df['flash_UV_OFF'][i][0][0:5])*3):
                flash_bgi2_best.extend([(df['flash_green_OFF_area'][i]-df['flash_UV_OFF_area'][i])/float(df['flash_green_OFF_area'][i]+df['flash_UV_OFF_area'][i])])
                flash_bgi_best.extend([(df['flash_green_OFF_peak'][i]-df['flash_UV_OFF_peak'][i])/float(df['flash_green_OFF_peak'][i]+df['flash_UV_OFF_peak'][i])])
            else:
                if df['flash_green_OFF_peak'][i] > (np.mean(df['flash_green_OFF'][i][0][0:5])+np.std(df['flash_green_OFF'][i][0][0:5])*3):
                    flash_bgi2_best.extend([1.])
                    flash_bgi_best.extend([1.])
                else :
                    flash_bgi2_best.extend([-1.])
                    flash_bgi_best.extend([-1.])
        else:
            if latency_label[i] == 2:#ON
                if df['flash_green_ON_peak'][i] > (np.mean(df['flash_green_ON'][i][0][0:5])+np.std(df['flash_green_ON'][i][0][0:5])*3) and df['flash_UV_ON_peak'][i] > (np.mean(df['flash_UV_ON'][i][0][0:5])+np.std(df['flash_UV_ON'][i][0][0:5])*3):
                    flash_bgi2_best.extend([(df['flash_green_ON_area'][i]-df['flash_UV_ON_area'][i])/float(df['flash_green_ON_area'][i]+df['flash_UV_ON_area'][i])])
                    flash_bgi_best.extend([(df['flash_green_ON_peak'][i]-df['flash_UV_ON_peak'][i])/float(df['flash_green_ON_peak'][i]+df['flash_UV_ON_peak'][i])])
                else:
                    if df['flash_green_ON_peak'][i] > (np.mean(df['flash_green_ON'][i][0][0:5])+np.std(df['flash_green_ON'][i][0][0:5])*3):
                        flash_bgi2_best.extend([1.])
                        flash_bgi_best.extend([1.])
                    else :
                        flash_bgi2_best.extend([-1.])      
                        flash_bgi_best.extend([-1.])   

            else:# latency_label[i] == 1:#ON/OFF
                flash_bgi2_best.extend([(df['flash_green_ON_area'][i]-df['flash_UV_ON_area'][i])/float(df['flash_green_ON_area'][i]+df['flash_UV_ON_area'][i])])
                flash_bgi_best.extend([(df['flash_green_ON_peak'][i]-df['flash_UV_ON_peak'][i])/float(df['flash_green_ON_peak'][i]+df['flash_UV_ON_peak'][i])])

                
                #---------------------------------------------------------------------------------------------------------------------------------------
        
    except: #if there was some problem, populate with np.nan
        print str('FAEL bgi: ') + df['Experiment'][i] + ' '+str(str(df['Cell'][i]))
        for d in [on_latency,off_latency,flash_luminance_class,
                  flash_bgi_dark,flash_bgi_bright,flash_bgi_best,flash_bgi2_dark,flash_bgi2_bright,flash_bgi2_best]:
            d.extend([np.nan])


df['ON_latency'] = on_latency#np.nan_to_num(np.array(on_latency))
df['OFF_latency'] =off_latency#np.nan_to_num(np.array(off_latency))
df['latency_label'] = latency_label
df['flash_luminance_class']=flash_luminance_class
df['flash_bgi_bright']=flash_bgi_bright
df['flash_bgi_dark']=flash_bgi_dark
df['flash_bgi_best']=flash_bgi_best
df['flash_bgi2_dark']=flash_bgi2_dark
df['flash_bgi2_bright']=flash_bgi2_bright
df['flash_bgi2_best']=flash_bgi2_best

###plot example cells from figure 2A

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

f,ax = plt.subplots(2,2,figsize=(9,4))
ymax = 155

#cell i
nwb_data = data['M179401']
cell = str(cell_numbers['M179401']['lgn_list_ex'][-1])
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[1::2],
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[0][0],color=Green,sparse=True,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[2::2],
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[1][0],color=Green,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[1::2]+0.016,
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[0][0],color=UV,sparse=True,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[2::2]+0.016,
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[1][0],color=UV,ymax=ymax)


#cell ii
nwb_data = data['M181423']
cell = str(101)#str(cell_numbers['M181423']['lgn_list_ex'][-10])
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[1::2],
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[0][1],color=Green,sparse=True,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[2::2],
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[1][1],color=Green,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[1::2]+0.016,
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[0][1],color=UV,sparse=True,ymax=ymax)
f=psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
            np.array(nwb_data['stimulus']['presentation']['flash_uv']['timestamps'])[2::2]+0.016,
            binsize=binsize,pre=pre,post=post,timeDomain=True,output='fig',
            axes=ax[1][1],color=UV,ymax=ymax)

ax[0][0].add_patch(patches.Rectangle((0,0),0.05,ymax,edgecolor='k',fill=None));
ax[1][0].add_patch(patches.Rectangle((0,0),0.05,ymax,color='k',alpha=0.5));
ax[0][1].add_patch(patches.Rectangle((0,0),0.05,ymax,edgecolor='k',fill=None));
ax[1][1].add_patch(patches.Rectangle((0,0),0.05,ymax,color='k',alpha=0.5));
for axis in ax.flatten():
    axis.set_xlim(-0.1,0.5)
plt.tight_layout()

###make scatter chart for figure 2B
####first, find the color that generates the peak response. then compare ON and OFF latency

In [ ]:
plt.figure()
plt.scatter(df['flash_green_ON_latency'],df['flash_green_OFF_latency'],edgecolor='k',facecolor=Green,s=40,alpha=0.6)
plt.scatter(df['flash_UV_ON_latency'],df['flash_UV_OFF_latency'],edgecolor='k',facecolor=UV,s=40,alpha=0.6,marker='o')
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('ON latency [msec]')
plt.ylabel('OFF latency [msec]')
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')

on_latency = []
off_latency = []
for i in range(np.shape(df)[0]):
    maxi = 0
    for stim in ['UV_OFF','UV_ON','green_OFF','green_ON']:
        if df['flash_'+stim+'_peak'][i] > maxi:
            maxi = df['flash_'+stim+'_peak'][i]
            max_type = stim

    if  df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i] > 30.:
        latency_label[i]=2
    else:
        if  df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i] < -30.:
            latency_label[i]=0
        else:
            if np.abs( df['flash_'+max_type.split('_')[0]+'_ON_latency'][i] - df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i]) < 30.:
                latency_label[i]=1
    on_latency.extend([df['flash_'+max_type.split('_')[0]+'_ON_latency'][i]])
    off_latency.extend([df['flash_'+max_type.split('_')[0]+'_OFF_latency'][i]])   
df['ON_latency'] = np.nan_to_num(np.array(on_latency))
df['OFF_latency'] =np.nan_to_num(np.array(off_latency))
# for i in range(len(latency_label)):
#     if np.abs(df['flash_green_OFF_latency'][i] - df['flash_green_ON_latency'][i]) < 16.:
#         latency_label[i]=1


plt.figure()
df['latency_label']=latency_label
plt.scatter(df['OFF_latency'][np.where(latency_label==0)[0]],df['ON_latency'][np.where(latency_label==0)[0]],edgecolor='k',facecolor='#000000',s=40,alpha=0.95)
plt.scatter(df['OFF_latency'][np.where(latency_label==1)[0]],df['ON_latency'][np.where(latency_label==1)[0]],edgecolor='k',facecolor='#808080',s=40,alpha=0.95)
plt.scatter(df['OFF_latency'][np.where(latency_label==2)[0]],df['ON_latency'][np.where(latency_label==2)[0]],edgecolor='k',facecolor='#FFFFFF',s=40,alpha=0.95)
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('ON latency [msec]')
plt.ylabel('OFF latency [msec]')
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')


plt.figure()
df['latency_label']=latency_label
plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label==0)[0]],df['flash_green_ON_latency'][np.where(latency_label==0)[0]],
            edgecolor=Green,facecolor='#000000',s=40,alpha=0.95)
plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label==1)[0]],df['flash_green_ON_latency'][np.where(latency_label==1)[0]],
            edgecolor=Green,facecolor='#808080',s=40,alpha=0.95)
plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label==2)[0]],df['flash_green_ON_latency'][np.where(latency_label==2)[0]],
            edgecolor=Green,facecolor='#FFFFFF',s=40,alpha=0.95)
plt.scatter(df['flash_UV_OFF_latency'][np.where(latency_label==0)[0]],df['flash_UV_ON_latency'][np.where(latency_label==0)[0]],
            edgecolor=UV,facecolor='#000000',s=40,alpha=0.95)
plt.scatter(df['flash_UV_OFF_latency'][np.where(latency_label==1)[0]],df['flash_UV_ON_latency'][np.where(latency_label==1)[0]],
            edgecolor=UV,facecolor='#808080',s=40,alpha=0.95)
plt.scatter(df['flash_UV_OFF_latency'][np.where(latency_label==2)[0]],df['flash_UV_ON_latency'][np.where(latency_label==2)[0]],
            edgecolor=UV,facecolor='#FFFFFF',s=40,alpha=0.95)
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('ON latency [msec]')
plt.ylabel('OFF latency [msec]')
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')


# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'ON', 'OFF', 'ON/OFF'
fracs = [np.shape(np.where(latency_label==2)[0])[0], np.shape(np.where(latency_label==0)[0])[0], np.shape(np.where(latency_label==1)[0])[0]]
colors = ['white', 'black', 'grey']
#explode=(0, 0.05, 0, 0)

patches_l, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[2].set_color('white')
autotexts[1].set_color('white')

#title('Raining Hogs and Dogs', bbox={'facecolor':'0.8', 'pad':5})
#plt.gcf().savefig(r'C:\Users\danield\OneDrive for Business\talks\20160202_neuralCoding\flash_latency_pie_kmeans_g.eps',format='eps')


### figure(s) 2DEF

In [ ]:
f,ax = plt.subplots(1,3,figsize=(6,2))
ax[0].hist(np.array(df['flash_bgi_best'][df['flash_luminance_class']=='ON']),range=(-1,1),bins=11)
ax[1].hist(np.array(df['flash_bgi_best'][df['flash_luminance_class']=='OFF']),range=(-1,1),bins=11)
ax[2].hist(np.array(df['flash_bgi_best'][df['flash_luminance_class']=='ON/OFF']),range=(-1,1),bins=11)
for axis in ax:
    axis.set_ylim(0,25)
    cleanAxes(axis,bottomLabels=True,leftLabels=True)
#plt.gcf().savefig(r'C:\Users\danield\OneDrive for Business\talks\20160202_neuralCoding\classification_color.eps',format='eps')

# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'UV', 'green', 'mixed'
df2 = df[df.latency_label==2]
fracs = [len(df2[df2['flash_bgi_best'] < -0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > 0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > -0.3][df2['flash_bgi_best'] < 0.3])/float(len(df2))]
colors = [UV, Green, '#FFFFFF']
#explode=(0, 0.05, 0, 0)

patches, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[0].set_color('white')
autotexts[1].set_color('white')

plt.gca().set_title('ON', bbox={'facecolor':'0.8', 'pad':5})
#plt.gcf().savefig(r'C:\Users\danield\OneDrive for Business\talks\20160202_neuralCoding\classification_color_ON.eps',format='eps')

# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'UV', 'green', 'mixed'
df2 = df[df.latency_label==0]
fracs = [len(df2[df2['flash_bgi_best'] < -0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > 0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > -0.3][df2['flash_bgi_best'] < 0.3])/float(len(df2))]
colors = [UV, Green, '#FFFFFF']
#explode=(0, 0.05, 0, 0)

patches, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[0].set_color('white')
autotexts[1].set_color('white')

plt.gca().set_title('OFF', bbox={'facecolor':'0.8', 'pad':5})
#plt.gcf().savefig(r'C:\Users\danield\OneDrive for Business\talks\20160202_neuralCoding\classification_color_OFF.eps',format='eps')

# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'UV', 'green', 'mixed'
df2 = df[df.latency_label==1]
fracs = [len(df2[df2['flash_bgi_best'] < -0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > 0.3])/float(len(df2)),
         len(df2[df2['flash_bgi_best'] > -0.3][df2['flash_bgi_best'] < 0.3])/float(len(df2))]
colors = [UV, Green, '#FFFFFF']
#explode=(0, 0.05, 0, 0)

patches, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[0].set_color('white')
autotexts[1].set_color('white')

plt.gca().set_title('ON/OFF', bbox={'facecolor':'0.8', 'pad':5})
#plt.gcf().savefig(r'C:\Users\danield\OneDrive for Business\talks\20160202_neuralCoding\classification_color_ONOFF.eps',format='eps')




<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 3</font></strong></div>

<br></br><div style="font-size:12pt;color:#000000"> load calculated spike-triggered average for each cell. each has been fit with 2D Gaussian, using fit_rf_2Dgauss_centerFixed() in color_units.py </div> 
<div style="font-size:9pt;color:#000000"> stas have been computed separately for each experiment, and pre-loaded here to save time.</div> 

In [ ]:
df_rf = pkl.load(open(os.path.join(pathtodata,'color_units_sandbox.pkl'),'rb'))

<div style="font-size:12pt;color:#000000"> example cell for figure 3A,B</div> 

<div style="font-size:12pt;color:#000000"> summary plot of receptive fields across space, pseudo-colored based on UVGi from flashes, and binned with means and standard errors along each axis</div> <div> Figure 3C</div> 

In [ ]:
df2 = df_rf[(np.abs(df_rf['rf_best_s_x']*5.)+np.abs(df_rf['rf_best_s_y']*5.))/2. < 16.] # limit only to reasonable receptive fields
df2 = df2[(df2['rf_best_x']-28)*4. > -20]
df2 = df2[(df2['rf_best_x']-28)*4. < 80]
df2 = df2[(df2['rf_best_y']-28)*4. < 80]
bins = 12
rang = (-20,80)

fig = plt.figure(figsize=(4,4))
ax_scatter =  placeAxesOnGrid(fig,dim=(1,1),xspan=(0,0.6),yspan=(0.4,1.0))
scatter_withcirclesize(ax_scatter,(df2['rf_best_x']-28)*4.,(df2['rf_best_y']-28)*4.,s=(np.abs(df2['rf_best_s_x']*5.)+np.abs(df2['rf_best_s_y']*5.))/2.,c=np.array(df2['flash_bgi_best']).tolist(),alpha=0.4,cmap=plt.cm.PRGn)
ax_scatter.set_xlim(rang[0],rang[1]);plt.xlabel('azimuth (deg)')
ax_scatter.set_ylim(rang[0],rang[1]);plt.ylabel('elevation (deg)')
ax_scatter.set_aspect('equal')

ax_azimuth =  placeAxesOnGrid(fig,dim=(1,1),xspan=(0,0.6),yspan=(0.0,0.37))
x_a=(df2['rf_best_x'][~np.isnan(df2['flash_bgi_best'])]-28)*4.
y_a=df2['flash_bgi_best'][~np.isnan(df2['flash_bgi_best'])]
n_a,_a=np.histogram(x_a,bins=bins,range=rang)
sy_a,_a=np.histogram(x_a,bins=bins,range=rang,weights=y_a)
sy2_a,_a=np.histogram(x_a,bins=bins,range=rang,weights=y_a*y_a)
mean_a = sy_a/n_a
std_a = (sy2_a/n_a- mean_a*mean_a)
ax_azimuth.errorbar((_a[1:]+_a[:-1])/2,mean_a,fmt='o-',color='k',yerr=std_a)
ax_azimuth.set_ylim(-1,1)
ax_azimuth.set_ylabel('UVGi')

ax_elevation =  placeAxesOnGrid(fig,dim=(1,1),xspan=(0.63,1),yspan=(0.4,1.0))
x_e=(df2['rf_best_y'][~np.isnan(df2['flash_bgi_best'])]-28)*4.
y_e=df2['flash_bgi_best'][~np.isnan(df2['flash_bgi_best'])]
n_e,_=np.histogram(x_e,bins=bins,range=rang)
sy_e,_e=np.histogram(x_e,bins=bins,range=rang,weights=y_e)
sy2_e,_e=np.histogram(x_e,bins=bins,range=rang,weights=y_e*y_e)
mean_e = sy_e/n_e
std_e = (sy2_e/n_e- mean_e*mean_e)
ax_elevation.errorbar(mean_e,(_e[1:]+_e[:-1])/2,fmt='o-',color='k',xerr=std_e)
ax_elevation.set_xlim(-1,1)
ax_elevation.set_xlabel('UVGi')
cleanAxes(ax_elevation)
cleanAxes(ax_azimuth)


<div style="font-size:12pt;color:#000000"> summary plot of receptive fields across space, pseudo-colored based on UVGi from flashes, limited to only strongly UV and strongly green preferring cells</div> <div> Figure 3D</div> 

In [ ]:
df2 = df_rf[(np.abs(df_rf['rf_best_s_x']*5.)+np.abs(df_rf['rf_best_s_y']*5.))/2. < 16.] # limit only to reasonable receptive field fits
df2 = df2[(df2['rf_best_x']-28)*4. > -20]
df2 = df2[(df2['rf_best_x']-28)*4. < 80]
df2 = df2[(df2['rf_best_y']-28)*4. < 80]

dfg = df2[df2['flash_bgi_best'] > 0.3] # get only the green units
dfu = df2[df2['flash_bgi_best'] < -0.3] # get only the UV units

bins = 12
rang = (-20,80)


fig = plt.figure(figsize=(6,6))
ax_scatter_g =  placeAxesOnGrid(fig,dim=(1,1),xspan=(0,0.45),yspan=(0.5,1.0))
ax_scatter_u =  placeAxesOnGrid(fig,dim=(1,1),xspan=(0.55,1.0),yspan=(0.5,1.0))

scatter_withcirclesize(ax_scatter_g,(dfg['rf_best_x']-28)*4.,(dfg['rf_best_y']-28)*4.,s=(np.abs(dfg['rf_best_s_x']*5.)+np.abs(dfg['rf_best_s_y']*5.))/2.,
                       c=np.array(dfg['flash_bgi_best']).tolist(),alpha=0.4,cmap=plt.cm.PRGn)
scatter_withcirclesize(ax_scatter_u,(dfu['rf_best_x']-28)*4.,(dfu['rf_best_y']-28)*4.,s=(np.abs(dfu['rf_best_s_x']*5.)+np.abs(dfu['rf_best_s_y']*5.))/2.,
                       c=np.array(dfu['flash_bgi_best']).tolist(),alpha=0.4,cmap=plt.cm.PRGn)


mean_x_g = np.mean((dfg['rf_best_x']-28)*4.)
std_x_g = np.std((dfg['rf_best_x']-28)*4.)
ste_x_g = std_x_g / np.sqrt(len(dfg['rf_best_x']))
mean_y_g = np.mean((dfg['rf_best_y']-28)*4.)
std_y_g = np.std((dfg['rf_best_y']-28)*4.)
ste_y_g = std_y_g/ np.sqrt(len(dfg['rf_best_y']))
ax_scatter_g.errorbar([mean_x_g],[mean_y_g],yerr=[std_y_g],xerr=[std_x_g],color=Green)

mean_x_u = np.mean((dfu['rf_best_x']-28)*4.)
std_x_u = np.std((dfu['rf_best_x']-28)*4.)
ste_x_u = std_x_u / np.sqrt(len(dfu['rf_best_x']))
mean_y_u = np.mean((dfu['rf_best_y']-28)*4.)
std_y_u = np.std((dfu['rf_best_y']-28)*4.)
ste_y_u = std_y_u/ np.sqrt(len(dfu['rf_best_y']))
ax_scatter_u.errorbar([mean_x_u],[mean_y_u],yerr=[std_y_u],xerr=[std_x_u],color=UV)

for ax_scatter in [ax_scatter_g,ax_scatter_u]:
    ax_scatter.set_xlim(rang[0],rang[1]);
    ax_scatter.set_ylim(rang[0],rang[1]);
    ax_scatter.set_aspect('equal')
ax_scatter_g.set_xlabel('azimuth (deg)')
ax_scatter_g.set_ylabel('elevation (deg)')
ax_scatter_g.set_title('green')
ax_scatter_u.set_title('UV')


<div> histogram of receptive field radius</div> <div> Figure 3E</div> 

In [ ]:
df2 = df_rf[(np.abs(df_rf['rf_best_s_x']*5.)+np.abs(df_rf['rf_best_s_y']*5.))/2. < 16.] # limit only to reasonable receptive fields
df2 = df2[(df2['rf_best_x']-28)*4. > -20]
df2 = df2[(df2['rf_best_x']-28)*4. < 80]
df2 = df2[(df2['rf_best_y']-28)*4. < 80]

dfu = df2[df2['flash_bgi_best'] < -0.3]
dfg = df2[df2['flash_bgi_best'] > 0.3]

plt.figure()
plt.hist((np.abs(df2['rf_best_s_x']*5.)+np.abs(df2['rf_best_s_y']*5.))/2.,bins=15,range=(0,15),normed=False,color='#FFFFFF')
#plt.hist(,bins=15,range=(0,15),normed=False)
plt.hist((np.array(np.abs(dfu['rf_best_s_x']*5.)+np.abs(dfu['rf_best_s_y']*5.))/2.,
         np.array(np.abs(dfg['rf_best_s_x']*5.)+np.abs(dfg['rf_best_s_y']*5.))/2.),
            bins=15,range=(0,15),normed=False,stacked=True,color=(UV,Green))
plt.xlabel('STRF radius',size=12)
plt.ylabel('cell count',size=12)
#plt.gca().set_aspect('equal')
plt.gca().tick_params(axis='both',labelsize=12)
cleanAxes(plt.gca(),bottomLabels=True,leftLabels=True)

<div> histogram of receptive field aspect ratios</div> <div> Figure 3F</div> 


In [ ]:
rf_aspect = []
for i in range(np.shape(df2)[0]):
    if np.abs(df_rf['rf_green_s_x'][i]) > np.abs(np.abs(df_rf['rf_green_s_y'][i])):
        rf_aspect.extend([np.abs(df_rf['rf_green_s_y'][i])/np.abs(df_rf['rf_green_s_x'][i])])
    else:
        rf_aspect.extend([np.abs(df_rf['rf_green_s_x'][i])/np.abs(df_rf['rf_green_s_y'][i])])
df2['rf_aspect']=rf_aspect

df2 = df_rf[(np.abs(df_rf['rf_best_s_x']*5.)+np.abs(df_rf['rf_best_s_y']*5.))/2. < 16.] # limit only to reasonable receptive fields
df2 = df2[(df2['rf_best_x']-28)*4. > -20]
df2 = df2[(df2['rf_best_x']-28)*4. < 80]
df2 = df2[(df2['rf_best_y']-28)*4. < 80]

dfu = df2[df2['flash_bgi_best'] < -0.3]
dfg = df2[df2['flash_bgi_best'] > 0.3]

plt.figure()
plt.hist(df2['rf_aspect'],bins=10,range=(0,1),normed=False,color='#FFFFFF')
plt.hist((np.array(dfu['rf_aspect']),
         np.array(dfg['rf_aspect'])),
            bins=10,range=(0,1),normed=False,stacked=True,color=(UV,Green))

plt.xlabel('STRF aspect',size=12)
plt.ylabel('cell count',size=12)
#plt.gca().set_aspect('equal')
plt.gca().tick_params(axis='both',labelsize=12)
cleanAxes(plt.gca(),bottomLabels=True,leftLabels=True)

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 4</font></strong></div>

<div> on and off latencies to green flashes</div> <div> Figure 4A</div> 

In [ ]:
stim_color = 'green'
latency_label_temp = np.zeros(np.shape(df)[0])-1
for i in range(np.shape(df)[0]):
    if  df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i] > 30.:
        latency_label_temp[i]=2
    else:
        if  df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i] < -30.:
            latency_label_temp[i]=0
        else:
            if np.abs( df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i]) < 30.:
                latency_label_temp[i]=1

plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label_temp==0)[0]],df['flash_green_ON_latency'][np.where(latency_label_temp==0)[0]],edgecolor='k',facecolor='#000000',s=40,alpha=0.95)
plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label_temp==1)[0]],df['flash_green_ON_latency'][np.where(latency_label_temp==1)[0]],edgecolor='k',facecolor='#808080',s=40,alpha=0.95)
plt.scatter(df['flash_green_OFF_latency'][np.where(latency_label_temp==2)[0]],df['flash_green_ON_latency'][np.where(latency_label_temp==2)[0]],edgecolor='k',facecolor='#FFFFFF',s=40,alpha=0.95)
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('ON latency [msec]',size=14)
plt.ylabel('OFF latency [msec]',size=14)
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')

# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'ON', 'OFF', 'ON/OFF'
fracs = [np.shape(np.where(latency_label_temp==2)[0])[0], np.shape(np.where(latency_label_temp==0)[0])[0],np.shape(np.where(latency_label_temp==1)[0])[0]]
colors = ['white', 'black', 'grey']
#explode=(0, 0.05, 0, 0)

patches, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[2].set_color('white')
autotexts[1].set_color('white')



<div> on and off latencies to UV flashes</div> <div> Figure 4B</div> 

In [ ]:
stim_color = 'UV'
latency_label_temp = np.zeros(np.shape(df)[0])-1
for i in range(np.shape(df)[0]):
    if  df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i] > 30.:
        latency_label_temp[i]=2
    else:
        if  df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i] < -30.:
            latency_label_temp[i]=0
        else:
            if np.abs( df['flash_'+stim_color+'_ON_latency'][i] - df['flash_'+stim_color+'_OFF_latency'][i]) < 30.:
                latency_label_temp[i]=1

plt.scatter(df['flash_'+stim_color+'_OFF_latency'][np.where(latency_label_temp==0)[0]],df['flash_'+stim_color+'_ON_latency'][np.where(latency_label_temp==0)[0]],edgecolor='k',facecolor='#000000',s=40,alpha=0.95)
plt.scatter(df['flash_'+stim_color+'_OFF_latency'][np.where(latency_label_temp==1)[0]],df['flash_'+stim_color+'_ON_latency'][np.where(latency_label_temp==1)[0]],edgecolor='k',facecolor='#808080',s=40,alpha=0.95)
plt.scatter(df['flash_'+stim_color+'_OFF_latency'][np.where(latency_label_temp==2)[0]],df['flash_'+stim_color+'_ON_latency'][np.where(latency_label_temp==2)[0]],edgecolor='k',facecolor='#FFFFFF',s=40,alpha=0.95)
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('ON latency [msec]',size=14)
plt.ylabel('OFF latency [msec]',size=14)
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')

# make a square figure and axes
plt.figure(figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'ON', 'OFF', 'ON/OFF'
fracs = [np.shape(np.where(latency_label_temp==2)[0])[0], np.shape(np.where(latency_label_temp==0)[0])[0],np.shape(np.where(latency_label_temp==1)[0])[0]]
colors = ['white', 'black', 'grey']
#explode=(0, 0.05, 0, 0)

patches, texts, autotexts = plt.pie(fracs, labels=labels,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
for text in texts:
    text.set_fontsize(20)
for text in autotexts:
    text.set_fontsize(24)
autotexts[2].set_color('white')
autotexts[1].set_color('white')



<div> green and uv latencies, for the appropriate sign flash</div> <div> Figure 4C</div> 
<br></br>
<div> difference between green and uv latencies, for appropriate sign flash</div> <div> Figure 4D</div> 

In [ ]:
plt.plot(df['flash_green_ON_latency'][np.where(df['latency_label']==0)[0]],df['flash_UV_ON_latency'][np.where(df['latency_label']==0)[0]],'o',color='#000000',alpha=1.0)
plt.plot(df['flash_green_OFF_latency'][np.where(df['latency_label']==2)[0]],df['flash_UV_OFF_latency'][np.where(df['latency_label']==2)[0]],'o',color='#FFFFFF',alpha=1.0)
plt.xlim(0,300);plt.ylim(0,300)
plt.xlabel('green latency (msec)',size=14)
plt.ylabel('UV latency (msec)',size=14)
plt.gca().set_aspect('equal')
plt.plot([0,300],[0,300],'r--')
cleanAxes(plt.gca())

plt.figure()
latency_diff_on = df['flash_green_ON_latency'][np.where(df['latency_label']==0)[0]] - df['flash_UV_ON_latency'][np.where(df['latency_label']==0)[0]]
latency_diff_off = df['flash_green_OFF_latency'][np.where(df['latency_label']==2)[0]]- df['flash_UV_OFF_latency'][np.where(df['latency_label']==2)[0]]
f = plt.hist((np.array(np.nan_to_num(latency_diff_on)),np.array(np.nan_to_num(latency_diff_off))),range=(-200,200),bins=20,color=('#000000','#FFFFFF'),stacked=False,rwidth=10)
plt.xlabel('green - UV latency (msec)',size=14)
plt.ylabel('cell count',size=14)
cleanAxes(plt.gca())

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 5</font></strong></div>
<br></br>
<div> extract some information from the color exchange experiments. for use in figures 5 and 6</div>


In [ ]:
        #---------------------------------------------------------------------------------------------------------------------------------------
        #calculate the PSTH for each cell, for each color, for increments and decrements 
        psth = psth_line(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                         np.array(nwb_data['stimulus']['presentation']['flash_green']['timestamps'])[1::2],
                         binsize=binsize,pre=pre,post=post,timeDomain=True,output='hist')

In [ ]:
exchange_class = []; exchange_class_fromplot = []; exchange_tsi = []
exchange_gdark = []; exchange_gbright = []; exchange_ubright = []; exchange_udark = []

for i in range(np.shape(df)[0]):        
    nwb_data = data[df['Experiment'][i]]
    cell = str(str(df['Cell'][i]))
    
    if  'color_exchange_uv' in nwb_data['stimulus']['presentation'].keys() or 'color_exchange_green' in nwb_data['stimulus']['presentation'].keys():
        mx=0
        if 'color_exchange_uv' in nwb_data['stimulus']['presentation'].keys():
            
            for k in all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'].keys():
                if np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][k][0]) > mx:
                    a=all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][-1.0]
                    transient = np.max(a[0][np.where(a[1]>0)[0][0]-1:np.where(a[1]>1.0)[0][0]+1])
                    sustained = np.mean(a[0][np.where(a[1]>0)[0][0]-1:np.where(a[1]>1.0)[0][0]+1])
                    tsi = (transient - sustained) / (transient + sustained)
                    mx =np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][k])
#             if all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][-1.0][0] == None:
#                 exchange_udark.extend([np.nan])
#                 exchange_ubright.extend([np.nan])
#             else:
#                 exchange_udark.extend([np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][-1.0][0][12:66])/mx])
#                 exchange_ubright.extend([np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][1.0][0][12:66]/mx)])
#         else:
#             pass
#             #exchange_udark.extend([np.nan])
#             #exchange_ubright.extend([np.nan])
            if all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_uv'][-1.0][0] == None:
                exchange_gdark.extend([np.nan])
                exchange_gbright.extend([np.nan])
            else:
                (green_responses_dark,green_responses_bright) = plotExchangeFromPSTH_2(all_data[df['Experiment'][i]],cell,'uv',dataonly=True)
                exchange_gdark.extend([np.mean(green_responses_dark[0:3])])
                exchange_gbright.extend([np.mean(green_responses_bright[0:3])])
                
        if 'color_exchange_green' in all_data[df['Experiment'][i]]['all']['lgn'][cell].keys(): 
            for k in all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'].keys():
                if np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][k]) > mx:
                    a=all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][-1.0]
                    transient = np.max(a[0][np.where(a[1]>0)[0][0]-1:np.where(a[1]>1.0)[0][0]+1])
                    sustained = np.mean(a[0][np.where(a[1]>0)[0][0]-1:np.where(a[1]>1.0)[0][0]+1])
                    tsi = (transient - sustained) / (transient + sustained)
                    mx =np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][k][0][12:66])
#             if all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][-1.0][0] == None:
#                 exchange_gdark.extend([np.nan])
#                 exchange_gbright.extend([np.nan])
#             else:
#                 exchange_gdark.extend([np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][-1.0][0][12:66])/mx])
#                 exchange_gbright.extend([np.max(all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][1.0][0][12:66])/mx])
#         else:
#             pass
#             #exchange_gdark.extend([np.nan])
#             #exchange_gbright.extend([np.nan])            
            if all_data[df['Experiment'][i]]['all']['lgn'][cell]['color_exchange_green'][-1.0][0] == None:
                exchange_udark.extend([np.nan])
                exchange_ubright.extend([np.nan])
            else:
                (uv_responses_dark,uv_responses_bright) = plotExchangeFromPSTH_2(all_data[df['Experiment'][i]],cell,'green',dataonly=True)
                exchange_udark.extend([np.mean(uv_responses_dark[0:3])])
                exchange_ubright.extend([np.mean(uv_responses_bright[0:3])])
        if exchange_gbright[-1] - exchange_ubright[-1] > 1.2:
            exchange_class_fromplot.extend(['gONuOFF'])
        else:
            if exchange_gdark[-1] - exchange_udark[-1] > 1.2:
                exchange_class_fromplot.extend(['gOFFuON'])
            else:
                if exchange_gdark[-1] - exchange_udark[-1] > -0.2 and (exchange_gdark[-1] - exchange_udark[-1]) - (exchange_gbright[-1] - exchange_ubright[-1]) > 0.2 :
                    exchange_class_fromplot.extend(['OFF'])
                else:
                    if exchange_gbright[-1] - exchange_ubright[-1] > -0.2 and (exchange_gbright[-1] - exchange_ubright[-1]) - (exchange_gdark[-1] - exchange_udark[-1]) > 0.2 :
                        exchange_class_fromplot.extend(['ON'])
                    else:
                        exchange_class_fromplot.extend(['ONOFF'])
                            
        if df['Cell'][i] in all_data[df['Experiment'][i]]['OFF_list']:
            exchange_class.extend(['OFF'])
        else:
            if df['Cell'][i] in all_data[df['Experiment'][i]]['ON_list']:
                exchange_class.extend(['ON'])
            else:
                if df['Cell'][i] in all_data[df['Experiment'][i]]['ONOFF_list']:
                    exchange_class.extend(['ONOFF'])  
                else:
                    if df['Cell'][i] in all_data[df['Experiment'][i]]['color_list']:
                        exchange_class.extend(['color'])
                    else:
                        exchange_class.extend([None])                     
        exchange_tsi.extend([tsi])
    else:
        exchange_class.extend([None])
        exchange_class_fromplot.extend([None])
        exchange_tsi.extend([None])
        exchange_gdark.extend([np.nan])
        exchange_gbright.extend([np.nan])
        exchange_ubright.extend([np.nan])
        exchange_udark.extend([np.nan])
        
df['exchange_class']=exchange_class
df['exchange_class_fromplot']=exchange_class_fromplot
df['exchange_tsi']=exchange_tsi
df['exchange_gdark']=exchange_gdark 
df['exchange_gbright']=exchange_gbright 
df['exchange_ubright']=exchange_ubright 
df['exchange_udark']=exchange_udark 

5A. this is a drawing, produced by hand elsewhere

In [ ]:
plt.imshow(os.path.join(pathtodata,'fig_5a.png')
cleanAxes(plt.gca(),total=True)

5B. Example non-opponent OFF cell in the color exchange protocol

In [ ]:
expt = 'M186118'
cell = '371'

5C. Example opponent green-off / uv-on cell in the color exchange protocol

In [ ]:
expt = 'M186118'
cell = '430'

5D and 5E. classification of responses from the plots in 5B and 5C

In [ ]:
bright_diff = df['exchange_gbright'] - df['exchange_ubright']
dark_diff = df['exchange_gdark'] - df['exchange_udark']

plt.figure()
plt.hist(bright_diff[np.isfinite(bright_diff)],range=(-2,2),bins=20)

plt.figure()
plt.hist(dark_diff[np.isfinite(dark_diff)],range=(-2,2),bins=20)


<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 6</font></strong></div>

examples, 6A

histogram, fig 6B

In [ ]:
plt.figure()
plt.hist((np.array(df[df['exchange_class']=='color']['exchange_tsi']),
         np.array(df[df['exchange_class']=='ON']['exchange_tsi']),
         np.array(df[df['exchange_class']=='OFF']['exchange_tsi']),
         np.array(df[df['exchange_class']=='ONOFF']['exchange_tsi']),
         ),histtype='barstacked',color=[Green,'#FFFFFF','#000000','#808080'],
         bins=25,range=(0,1),normed=False)
plt.xlabel('transient - sustained index',size=12)
plt.ylabel('portion of cells',size=12)
plt.gca().tick_params(axis='both',labelsize=12)

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 7</font></strong></div>

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 8</font></strong></div>

In [ ]:
unit = '394'
fig = plt.figure()
plotExchangeFromPSTH_2(M186118,unit,'uv',axis=plt.gca())
plotExchangeFromPSTH_2(M186118,unit,'green',axis=plt.gca())
fig = plt.figure()
plotExchangeFromPSTH(M186118,unit,'uv',axis=plt.gca())
plotExchangeFromPSTH(M186118,unit,'green',axis=plt.gca())

In [ ]:
unit = '371'
fig = plt.figure()
plotExchangeFromPSTH_2(M186118,unit,'uv',axis=plt.gca())
plotExchangeFromPSTH_2(M186118,unit,'green',axis=plt.gca())
fig = plt.figure()
plotExchangeFromPSTH(M186118,unit,'uv',axis=plt.gca())
plotExchangeFromPSTH(M186118,unit,'green',axis=plt.gca())

In [ ]:
for i,data in enumerate([M186118,M186100,M180417,M186098]):
    lgn_units = data['lgn_list_ex']
    for u in lgn_units:
        unit = str(u)
        fig,ax=plt.subplots(1,2)
        plotExchangeFromPSTH_2(data,unit,'uv',axis=ax[0])
        plotExchangeFromPSTH_2(data,unit,'green',axis=ax[0])
        plotExchangeFromPSTH(data,unit,'uv',axis=ax[1])
        plotExchangeFromPSTH(data,unit,'green',axis=ax[1])
        plt.title(data['name']+' '+str(unit))

In [ ]:
data =M186118
unit = '371'
summarize_color(data,str(unit))

In [ ]:
for i,data in enumerate([M186118,M186100,M180417,M186098]):
    lgn_units = data['lgn_list_ex']
    for u in lgn_units:
        if not os.path.isfile(os.path.join(r'C:\Users\danield\OneDrive for Business\allenTransfer\notebooks\ephys\color_units_plots\eps',data['name']+'_'+str(u)+'.eps')):
            try:
                summarize_color(data,str(u))
            except:
                pass
        plt.close('all')

<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<br></br>
<div style="font-size:16pt;color:#000000"><strong><font size='20' color='red'>Figure 9</font></strong></div>

<div style="font-size:12pt;color:#000000">these data are taken from analysis of local field potentials and the calcualted current source potentials from these potentials. </div> 
<div style="font-size:12pt;color:#000000">the structure is: [(total cortical distance covered, ), (span of green source), (span of uv source)]] </div> 

In [ ]:
f_180417 = [(800,1.58),(0.617,0.8397),(0.6873,0.9122)]
f_186101 = [(800,2.5136),(0.7566,1.57),(0.8974,1.7421)]
f_186100 = [(800,2.545),(1.1368,1.888),(1.1092,1.9691)]
f_186098 = [(800,3.2),(0.8626,1.9459),(0.9825,2.031)]
f_192079 = [(1000,1.94),(0.5242,0.99),(0.68,1.1955)]

In [ ]:
from matplotlib import patches

green_top=[];uv_top=[];green_bottom=[];uv_bottom=[]
plt.figure()
for i,exp in enumerate([f_180417,f_186101,f_186100,f_186098,f_192079]):
    microns_per_unit = exp[0][0]/float(exp[0][1])
    green = np.array(exp[1])* microns_per_unit
    uv = np.array(exp[2])* microns_per_unit
    green_top.extend([green[0]])
    green_bottom.extend([green[1]])
    uv_top.extend([uv[0]])
    uv_bottom.extend([uv[1]])
    plt.gca().add_patch(patches.Rectangle((i,green[1]),
                                         0.4,
                                         -1*(green[1]-green[0]),
                                         fill=True,
                                         facecolor=Green,
                                         alpha=0.8))
    plt.gca().add_patch(patches.Rectangle((i+0.45,uv[1]),
                                         0.4,
                                         -1*(uv[1]-uv[0]),
                                         fill=True,
                                         facecolor=UV,
                                         alpha=0.8))
    print 'green: '+str(green[0])+' - '+str(green[1])+'   UV: '+str(uv[0])+' - '+str(uv[1])
plt.ylim(800,0)
plt.xlim(0,5)
ephys.cleanAxes(plt.gca(),leftLabels=True)
plt.gcf().savefig('/Users/administrator/Desktop/l4sinks.eps',format='eps')

In [ ]:
ephys.Welchs(green_top,uv_top)

In [ ]:
green_middle = (np.array(green_top)+np.array(green_bottom)) / 2.
uv_middle = (np.array(uv_top)+np.array(uv_bottom)) / 2.
ephys.Welchs(green_middle,uv_middle)

In [ ]:
sta_green = []; sta_UV=[]; sta_green_fit = []; sta_UV_fit = [];
backup_center=(34,26)
for i in range(np.shape(df)[0]):
    try:
        nwb_data = data[df['Experiment'][i]]
        cell = str(str(df['Cell'][i]))
        sys.stdout.write('\% complete: %s' % str(i*100/np.shape(df)[0]));sys.stdout.flush() # show % progress

        #calculate spike-triggered average
        sta_g = sta(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                              np.array(nwb_data['stimulus']['presentation']['binary_green']['data']).T,
                              np.array(nwb_data['stimulus']['presentation']['binary_green']['timestamps']),
                              taus=(np.linspace(-110,580,70)),samplingRateInkHz=0.001)#sampling rate scales the tau values to seconds
        sta_green.extend([sta_g]);
        sta_u = sta(np.array(nwb_data['processing']['LGN']['UnitTimes'][cell]['times']),
                              np.array(nwb_data['stimulus']['presentation']['binary_uv']['data']).T,
                              np.array(nwb_data['stimulus']['presentation']['binary_uv']['timestamps']),
                              taus=(np.linspace(-110,580,70)),samplingRateInkHz=0.001)#sampling rate scales the tau values to seconds
        sta_UV.extend([sta_u]);

        #fit the results with a 2D Gaussian
        sta_green_fit.extend([fitRF(sta_green[-1],backup_center=backup_center,rfsizeguess=1.5,centerfixed=True)]) 
        sta_UV_fit.extend([fitRF(sta_UV[-1],backup_center=backup_center,rfsizeguess=1.5,centerfixed=True)])

        if sta_green_fit[-1]['roughquality']=='good':
            backup_center = sta_green_fit[-1]['center']
        
    except:
        print df['Experiment'][i]+':  '+cell
        for measure in [sta_green,sta_UV,sta_green_fit,sta_UV_fit]:
            measure.extend([np.nan])
        print str('FAEL RF: ') + df['Experiment'][i] + ' '+str(str(df['Cell'][i]))

df['sta_green']=sta_green;df['sta_UV']=sta_UV;
df['sta_green_fit']=sta_green_fit;df['sta_UV_fit']=sta_UV_fit;

df_rf = df=pkl.load(open('/Volumes/DATA1/ephys/color_units_sandbox.pkl','rb'))